# Algoritmo de Sobel

## 1. Sustento Matemático
El operador de Sobel calcula una aproximación discreta del **gradiente** de la intensidad de una imagen. Este método está fundamentado en el concepto de **diferencias finitas** para aproximar las derivadas parciales de la función de imagen.

### 1.1 Definición del Gradiente
Para una función continua $f(x, y)$, el gradiente se define como un vector que apunta en la dirección de mayor cambio de intensidad:

$$\nabla f = \begin{bmatrix} \frac{\partial f}{\partial x} \\ \frac{\partial f}{\partial y} \end{bmatrix}$$

### 1.2 Magnitud del Gradiente
La magnitud del gradiente representa la "fuerza" del borde y se calcula mediante la norma del vector:

$$|\nabla f| = \sqrt{\left( \frac{\partial f}{\partial x} \right)^2 + \left( \frac{\partial f}{\partial y} \right)^2}$$

## 2. Relación con Métodos Numéricos: Diferencias Finitas

### 2.1 Aproximación de Derivadas
El método de Sobel utiliza **diferencias finitas ponderadas**, una técnica clásica de métodos numéricos para aproximar derivadas cuando solo disponemos de datos discretos (como los píxeles de una imagen).

* **Derivada Parcial en X (dirección horizontal):**
    Representa el cambio de intensidad a lo largo de las columnas:
    $$\frac{\partial f}{\partial x} \approx G_x = \begin{bmatrix} -1 & 0 & +1 \\ -2 & 0 & +2 \\ -1 & 0 & +1 \end{bmatrix} * I$$

* **Derivada Parcial en Y (dirección vertical):**
    Representa el cambio de intensidad a lo largo de las filas:
    $$\frac{\partial f}{\partial y} \approx G_y = \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ +1 & +2 & +1 \end{bmatrix} * I$$

### 2.2 Interpretación como Operador de Diferenciación
| Aspecto Numérico | Aplicación en Sobel |
| :--- | :--- |
| **Diferencias centrales** | Máscara con valores $-1, 0, +1$ |
| **Suavizado (regularización)** | Coeficientes $1, 2, 1$ (promedio ponderado) |
| **Convolución discreta** | Operador aplicado sobre vecindad $3 \times 3$ |

## 3. Pseudocódigo del Algoritmo

### ALGORITMO SOBEL(I: imagen de entrada)

**ENTRADA:** $I[m][n]$ (matriz de intensidades en escala de grises)  
**SALIDA:** $G[m][n]$ (Magnitud del gradiente), $\theta[m][n]$ (Dirección del gradiente)

**PASOS:**

1. **INICIALIZAR:** Matrices $G_x, G_y, G, \theta$ con ceros (del mismo tamaño que $I$).
2. **DEFINIR** máscaras de convolución:

   **Máscara X:**
   $$\begin{bmatrix} -1 & 0 & +1 \\ -2 & 0 & +2 \\ -1 & 0 & +1 \end{bmatrix}$$

   **Máscara Y:**
   $$\begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ +1 & +2 & +1 \end{bmatrix}$$

3. **PARA** cada píxel $(i, j)$ donde $1 \leq i \leq m-2$ y $1 \leq j \leq n-2$:
   
   a. **Extraer** vecindad $3 \times 3$ centrada en $(i, j)$: $V = I[i-1:i+2, j-1:j+2]$
   
   b. **Calcular** gradiente horizontal: $G_x[i][j] = \sum (V \odot MascaraX)$
   
   c. **Calcular** gradiente vertical: $G_y[i][j] = \sum (V \odot MascaraY)$
   
   d. **Calcular** magnitud: $G[i][j] = \sqrt{G_x[i][j]^2 + G_y[i][j]^2}$
   
   e. **Calcular** dirección (opcional): $\theta[i][j] = \arctan2(G_y[i][j], G_x[i][j])$

4. **NORMALIZAR** $G$ al rango $[0, 255]$ si es necesario.
5. **RETORNAR** $G, \theta$

## 4. Desarrollo Teórico (Series de Taylor)

Para fundamentar el error de aproximación, analizamos las expansiones de Taylor:

### Para Diferencias Progresivas
$$f(x+h) = f(x) + hf'(x) + \frac{h^2}{2}f''(x) + O(h^3)$$

**Despejando:**
$$f'(x) = \frac{f(x+h) - f(x)}{h} - \frac{h}{2}f''(x) + O(h^2)$$
$$\underbrace{Error: O(h)}_{\text{Primer orden}}$$

### Para Diferencias Regresivas
$$f(x-h) = f(x) - hf'(x) + \frac{h^2}{2}f''(x) - O(h^3)$$

**Despejando:**
$$f'(x) = \frac{f(x) - f(x-h)}{h} + \frac{h}{2}f''(x) + O(h^2)$$
$$\underbrace{Error: O(h)}_{\text{Primer orden}}$$

### Para Diferencias Centrales (Sobel)
Restando las expansiones de Taylor anteriores:
$$f(x+h) - f(x-h) = 2hf'(x) + O(h^3)$$

**Despejando:**
$$f'(x) = \frac{f(x+h) - f(x-h)}{2h} + O(h^2)$$
$$\underbrace{Error: O(h^2)}_{\text{Segundo orden (Más preciso)}}$$


## 5. Comparación de Operadores

Dependiendo del método numérico elegido para aproximar la derivada, las máscaras (kernels) resultantes cambian su estructura y precisión:

### Sobel con Diferencias Regresivas (No estándar)
Esta versión utiliza una aproximación hacia atrás. Aunque es computacionalmente sencilla, no es la más utilizada porque tiende a desplazar visualmente la posición de los bordes en la imagen:

$$\begin{bmatrix} -1 & 1 & 0 \\ -2 & 2 & 0 \\ -1 & 1 & 0 \end{bmatrix}$$

### Sobel Estándar (Diferencias Centrales)
Es el estándar de la industria y el que se implementa en la mayoría de las librerías. Al usar diferencias centrales, logra una mejor simetría y un error de segundo orden $O(h^2)$:

**Gradiente en X ($G_x$):**
$$G_x = \begin{bmatrix} -1 & 0 & +1 \\ -2 & 0 & +2 \\ -1 & 0 & +1 \end{bmatrix}$$

**Gradiente en Y ($G_y$):**
$$G_y = \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ +1 & +2 & +1 \end{bmatrix}$$

## 6. Interpretación Geométrica

Desde una perspectiva geométrica, el algoritmo de Sobel no solo detecta cambios, sino que interpreta la imagen como una **superficie continua de intensidades**, donde cada píxel tiene una "altura" basada en su valor de gris.

### 1. El Plano Tangente
Al calcular $G_x$ y $G_y$, el algoritmo está estimando la inclinación de un **plano tangente** a la superficie de la imagen en el punto $(i, j)$. 
* $G_x$ representa la pendiente del plano en dirección este-oeste.
* $G_y$ representa la pendiente del plano en dirección norte-sur.

### 2. El Vector Gradiente
El vector $\nabla f = (G_x, G_y)$ es perpendicular a las líneas de contorno de la imagen. 
* Su **dirección** $\theta$ apunta hacia donde la intensidad crece más rápidamente.
* Su **magnitud** $|G|$ indica qué tan "empinada" es la pendiente. Un borde es, simplemente, una zona con una pendiente muy pronunciada.

### 3. Visualización de Bordes
En la práctica, los bordes detectados corresponden a los lugares donde la superficie de intensidad tiene su máxima tasa de cambio. Geométricamente, el operador de Sobel actúa como un **filtro de paso alto**, eliminando las zonas planas (frecuencias bajas) y resaltando las transiciones abruptas (frecuencias altas).

In [ ]:
%%html
<div id="jup-container" style="position: relative; width: 100%; height: 550px; background: #000; overflow: hidden; border-radius: 8px;">
    <div id="ui" style="position: absolute; top: 10px; left: 10px; z-index: 10; background: rgba(0,0,0,0.6); padding: 10px; border-radius: 10px; display: flex; flex-wrap: wrap; gap: 5px;">
        <button onclick="toggle('glasses')" style="cursor:pointer; padding: 5px 10px;">👓 Lentes</button>
        <button onclick="toggle('mustache')" style="cursor:pointer; padding: 5px 10px;">🥸 Bigote</button>
        <button onclick="toggle('hat')" style="cursor:pointer; padding: 5px 10px;">🎩 Sombrero</button>
        <button onclick="toggle('mole')" style="cursor:pointer; padding: 5px 10px;">🟤 Lunar</button>
        <button onclick="toggle('points')" style="cursor:pointer; padding: 5px 10px;">🟢 FaceMesh</button>
        <button onclick="toggle('sobel')" style="cursor:pointer; padding: 5px 10px; background: #ffeb3b; border: none; font-weight: bold;">🧠 Sobel</button>
    </div>

    <video id="video" style="display: none;" autoplay muted playsinline></video>
    <canvas id="canvas" style="width: 100%; height: 100%; object-fit: cover;"></canvas>
</div>

<script src="https://cdn.jsdelivr.net/npm/@mediapipe/face_mesh/face_mesh.js"></script>
<script src="https://cdn.jsdelivr.net/npm/@mediapipe/camera_utils/camera_utils.js"></script>

<script>
(function() {
    const video = document.getElementById("video");
    const canvas = document.getElementById("canvas");
    const container = document.getElementById("jup-container");
    const ctx = canvas.getContext("2d");

    function resize(){
        canvas.width = container.clientWidth;
        canvas.height = container.clientHeight;
    }
    resize();
    window.addEventListener('resize', resize);

    const filters = {
        glasses: false, mustache: false, hat: false,
        mole: false, points: false, sobel: false
    };

    window.toggle = function(f) {
        filters[f] = !filters[f];
    };

    /* ========= FACEMESH ========= */
    const faceMesh = new FaceMesh({
        locateFile: f => `https://cdn.jsdelivr.net/npm/@mediapipe/face_mesh/${f}`
    });

    faceMesh.setOptions({
        maxNumFaces: 1,
        refineLandmarks: true,
        staticImageMode: false
    });

    faceMesh.onResults(drawResults);

    function drawResults(res){
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        ctx.drawImage(res.image, 0, 0, canvas.width, canvas.height);

        if(res.multiFaceLandmarks && res.multiFaceLandmarks.length > 0){
            const f = res.multiFaceLandmarks[0];

            if(filters.points){
                ctx.fillStyle = "lime";
                f.forEach(p => {
                    ctx.beginPath();
                    ctx.arc(p.x * canvas.width, p.y * canvas.height, 2, 0, Math.PI * 2);
                    ctx.fill();
                });
            }
            drawFilters(f);
        }

        if(filters.sobel) applySobel();
    }

    /* ========= FILTROS (Lógica Matemática) ========= */
    function drawFilters(f){
        const le=f[33], re=f[263], lip=f[13], cheek=f[50];
        const cx=(le.x+re.x)/2*canvas.width;
        const cy=(le.y+re.y)/2*canvas.height;
        const d=Math.abs(le.x-re.x)*canvas.width;

        if(filters.glasses){
            ctx.strokeStyle="black"; ctx.lineWidth=4;
            ctx.strokeRect(cx-d*0.7,cy-d*0.25,d*0.6,d*0.35);
            ctx.strokeRect(cx+d*0.1,cy-d*0.25,d*0.6,d*0.35);
            ctx.beginPath(); ctx.moveTo(cx-d*0.1,cy); ctx.lineTo(cx+d*0.1,cy); ctx.stroke();
        }

        if(filters.mustache){
            const mx=lip.x*canvas.width;
            const my=lip.y*canvas.height-d*0.15;
            ctx.strokeStyle="#2a1a12"; ctx.lineWidth=d*0.18; ctx.lineCap="round";
            ctx.beginPath();
            ctx.moveTo(mx,my);
            ctx.bezierCurveTo(mx-d*0.3,my-d*0.2,mx-d,my+d*0.3,mx-d*1.1,my+d*0.4);
            ctx.moveTo(mx,my);
            ctx.bezierCurveTo(mx+d*0.3,my-d*0.2,mx+d,my+d*0.3,mx+d*1.1,my+d*0.4);
            ctx.stroke();
        }

        if(filters.mole){
            ctx.fillStyle="#3b2b1f"; ctx.beginPath();
            ctx.arc(cheek.x*canvas.width+12,cheek.y*canvas.height,5,0,Math.PI*2);
            ctx.fill();
        }

        if(filters.hat){
            ctx.fillStyle="#6b4a2b";
            ctx.fillRect(cx-d*1.2,cy-d*1.25,d*2.4,d*0.35);
            ctx.fillRect(cx-d*0.5,cy-d*1.9,d,d*0.65);
        }
    }

    /* ========= SOBEL (Algoritmo de Convolución) ========= */
    function applySobel(){
        const img = ctx.getImageData(0, 0, canvas.width, canvas.height);
        const d = img.data, w = img.width;
        const gx = [-1, 0, 1, -2, 0, 2, -1, 0, 1];
        const gy = [-1, -2, -1, 0, 0, 0, 1, 2, 1];
        const out = new Uint8ClampedArray(d.length);

        for(let i=0; i<d.length; i+=4){
            let sx=0, sy=0;
            for(let k=0; k<9; k++){
                const col = (k % 3) - 1;
                const row = Math.floor(k / 3) - 1;
                const p = i + (col * 4) + (row * w * 4);
                if(p < 0 || p >= d.length) continue;
                const gray = d[p] * 0.3 + d[p+1] * 0.59 + d[p+2] * 0.11;
                sx += gray * gx[k];
                sy += gray * gy[k];
            }
            const m = Math.min(255, Math.sqrt(sx*sx + sy*sy));
            out[i] = out[i+1] = out[i+2] = m;
            out[i+3] = 255;
        }
        img.data.set(out);
        ctx.putImageData(img, 0, 0);
    }

    /* ========= INICIO DE CÁMARA ========= */
    const camera = new Camera(video, {
        onFrame: async () => {
            await faceMesh.send({ image: video });
        },
        width: 640,
        height: 480
    });

    navigator.mediaDevices.getUserMedia({video: true})
        .then(stream => {
            video.srcObject = stream;
            camera.start();
        })
        .catch(err => {
            console.error("Error al acceder a la cámara: ", err);
            alert("No se pudo acceder a la cámara. Asegúrate de dar permisos en el navegador.");
        });
})();
</script>

# Algoritmo de Canny (Canny Edge Detector)

## 1. Sustento Matemático (Criterios de Optimización)
A diferencia de otros operadores, el detector de bordes de Canny se diseñó como un problema de optimización matemática. John Canny estableció tres criterios fundamentales para considerar una detección de bordes como "óptima":

### 1.1 Criterios de Calidad
* **Baja Tasa de Error (Detección):** El algoritmo debe marcar tantos bordes reales como sea posible y evitar falsos positivos causados por el ruido.
* **Buena Localización:** La distancia entre el píxel marcado como borde y el centro del borde real debe ser mínima.
* **Respuesta Única:** Un solo borde real no debe generar múltiples bordes detectados (evitar el "efecto de doble borde").

### 1.2 Formulación Matemática
Para lograr esto, se busca una función $f(x)$ que maximice el producto de la relación señal-ruido ($SNR$) y la localización ($L$):

$$SNR = \frac{|\int_{-W}^{W} G(-x)f(x)dx|}{\sigma \sqrt{\int_{-W}^{W} f^2(x)dx}}$$

$$L = \frac{|\int_{-W}^{W} G'(-x)f'(x)dx|}{\sigma \sqrt{\int_{-W}^{W} (f')^2(x)dx}}$$

Donde:
* $G(x)$ representa la señal del borde.
* $f(x)$ es el filtro aplicado.
* $\sigma$ es la desviación estándar del ruido.

# Algoritmo de Canny (Canny Edge Detector)

## 1. Sustento Matemático (Criterios de Optimización)
A diferencia de otros operadores, el detector de bordes de Canny se diseñó como un problema de optimización matemática. John Canny estableció tres criterios fundamentales para considerar una detección de bordes como "óptima":

### 1.1 Criterios de Calidad
* **Baja Tasa de Error (Detección):** El algoritmo debe marcar tantos bordes reales como sea posible y evitar falsos positivos causados por el ruido.
* **Buena Localización:** La distancia entre el píxel marcado como borde y el centro del borde real debe ser mínima.
* **Respuesta Única:** Un solo borde real no debe generar múltiples bordes detectados (evitar el "efecto de doble borde").

### 1.2 Formulación Matemática
Para lograr esto, se busca una función $f(x)$ que maximice el producto de la relación señal-ruido ($SNR$) y la localización ($L$):

$$SNR = \frac{|\int_{-W}^{W} G(-x)f(x)dx|}{\sigma \sqrt{\int_{-W}^{W} f^2(x)dx}}$$

$$L = \frac{|\int_{-W}^{W} G'(-x)f'(x)dx|}{\sigma \sqrt{\int_{-W}^{W} (f')^2(x)dx}}$$

Donde:
* $G(x)$ representa la señal del borde.
* $f(x)$ es el filtro aplicado.
* $\sigma$ es la desviación estándar del ruido.

## 2. Relación con Métodos Numéricos: Regularización

### 2.1 Suavizado Gaussiano
Dado que el cálculo de derivadas es una operación que amplifica el ruido (frecuencias altas), el algoritmo de Canny aplica una etapa de **suavizado** o **regularización** previa. Esto se basa en la convolución de la imagen original $I$ con un núcleo (kernel) Gaussiano.

La función Gaussiana en 2D se define como:

$$G(x, y) = \frac{1}{2\pi\sigma^2} e^{-\frac{x^2 + y^2}{2\sigma^2}}$$

### 2.2 Discretización del Kernel
En métodos numéricos, aproximamos esta función continua mediante una máscara discreta. Por ejemplo, para un $\sigma = 1.4$, un kernel $5 \times 5$ aproximado es:

$$K = \frac{1}{273} \begin{bmatrix} 1 & 4 & 7 & 4 & 1 \\ 4 & 16 & 26 & 16 & 4 \\ 7 & 26 & 41 & 26 & 7 \\ 4 & 16 & 26 & 16 & 4 \\ 1 & 4 & 7 & 4 & 1 \end{bmatrix}$$

### 2.3 Operación de Convolución
La imagen suavizada $I_{suave}$ se obtiene mediante la suma ponderada en la vecindad del píxel:

$$I_{suave}(i, j) = \sum_{m=-k}^{k} \sum_{n=-k}^{k} I(i-m, j-n) \cdot K(m, n)$$

> **Importante:** Esta etapa reduce el error en la aproximación de las derivadas parciales que se calcularán en el siguiente paso.

## 3. Cálculo de Gradiente y Dirección

Una vez suavizada la imagen, el algoritmo de Canny utiliza operadores de diferenciación (como Sobel) para encontrar las variaciones de intensidad en las direcciones $x$ e $y$.

### 3.1 Componentes del Gradiente
Se aplican las máscaras de diferencias finitas para obtener las derivadas parciales:

* **Gradiente Horizontal:** $G_x = M_x * I_{suave}$
* **Gradiente Vertical:** $G_y = M_y * I_{suave}$

### 3.2 Magnitud y Orientación
A partir de estas componentes, se calcula la magnitud total del cambio y la dirección del vector gradiente:

* **Magnitud del Gradiente:**
    $$M(i, j) = \sqrt{G_x(i, j)^2 + G_y(i, j)^2}$$

* **Ángulo de Orientación:**
    $$\alpha(i, j) = \arctan\left(\frac{G_y(i, j)}{G_x(i, j)}\right)$$

### 3.3 Cuantización de Direcciones
Para el procesamiento numérico posterior, el ángulo $\alpha$ se redondea a uno de los cuatro sectores de una vecindad $3 \times 3$:

| Ángulo Aproximado | Dirección del Borde |
| :--- | :--- |
| **$0^\circ$** | Horizontal |
| **$45^\circ$** | Diagonal Positiva |
| **$90^\circ$** | Vertical |
| **$135^\circ$** | Diagonal Negativa |

## 4. Supresión de No Máximos (Non-Maximum Suppression)

Este proceso es una técnica de **adelgazamiento de bordes**. Su objetivo es eliminar los píxeles que no forman parte de la cresta del gradiente, dejando únicamente los máximos locales.

### 4.1 Lógica Algorítmica
Para cada píxel $(i, j)$, el algoritmo compara su magnitud $M(i, j)$ con las magnitudes de los dos píxeles vecinos en la dirección del gradiente $\alpha(i, j)$.

* Si $M(i, j)$ es la mayor de las tres, el píxel se conserva.
* Si no es la mayor, se suprime asignándole un valor de $0$.

### 4.2 Esquema de Comparación según la Dirección
Dependiendo del ángulo cuantizado, se eligen los vecinos específicos:

| Ángulo ($\alpha$) | Vecino 1 | Vecino 2 | Eje de Comparación |
| :--- | :---: | :---: | :--- |
| **$0^\circ$** | $(i, j+1)$ | $(i, j-1)$ | Horizontal |
| **$45^\circ$** | $(i-1, j+1)$ | $(i+1, j-1)$ | Diagonal Positiva |
| **$90^\circ$** | $(i-1, j)$ | $(i+1, j)$ | Vertical |
| **$135^\circ$** | $(i-1, j-1)$ | $(i+1, j+1)$ | Diagonal Negativa |

### 4.3 Resultado Numérico
El resultado de esta operación es una imagen de bordes delgados (frecuentemente llamada *Thin Edges*), definida matemáticamente como:

$$M_{thin}(i, j) = \begin{cases} M(i, j) & \text{si } M(i, j) > M_{vecinos} \\ 0 & \text{en otro caso} \end{cases}$$

## 5. Umbralización con Histéresis

Este paso determina qué bordes son genuinos y cuáles deben descartarse. A diferencia de una umbralización simple, Canny utiliza dos niveles de decisión para mantener la conectividad de los bordes.

### 5.1 Definición de Umbrales
Se establecen dos valores críticos:
* **$T_{high}$ (Umbral Superior):** Define los bordes de alta confianza.
* **$T_{low}$ (Umbral Inferior):** Permite la inclusión de bordes más débiles siempre que tengan sustento estructural.

### 5.2 Clasificación de Píxeles
Para cada píxel de la imagen resultante del paso anterior, se aplica la siguiente lógica:

| Categoría | Condición de Magnitud ($M$) | Decisión Final |
| :--- | :--- | :--- |
| **Borde Fuerte** | $M \geq T_{high}$ | Se conserva (Píxel de borde definitivo) |
| **Borde Débil** | $T_{low} \leq M < T_{high}$ | Se conserva **solo si** está conectado a uno fuerte |
| **No-Borde** | $M < T_{low}$ | Se elimina (Valor 0) |

### 5.3 Análisis de Conectividad (Hysteresis)
Matemáticamente, un píxel débil $W$ en la posición $(i, j)$ se convierte en borde si existe un píxel fuerte $S$ en su vecindad de Moore ($8$ vecinos):

$$B(i, j) = \begin{cases} 1 & \text{si } M(i, j) \geq T_{high} \\ 1 & \text{si } T_{low} \leq M(i, j) < T_{high} \text{ y } \exists \text{ fuerte en vecindad} \\ 0 & \text{en otro caso} \end{cases}$$

> **Ventaja:** Este método evita que un borde se "rompa" debido a pequeñas variaciones de intensidad a lo largo de su trayectoria.

## 6. Pseudocódigo del Algoritmo de Canny

**ALGORITMO CANNY_EDGE_DETECTOR($I$, $\sigma$, $T_{low}$, $T_{high}$)**

**ENTRADA:** * $I$: Imagen original en escala de grises.  
* $\sigma$: Desviación estándar para el suavizado Gaussiano.  
* $T_{low}, T_{high}$: Umbrales para la histéresis.

**SALIDA:** * $E$: Imagen binaria con los bordes detectados.

**PASOS:**

1. **REDUCCIÓN DE RUIDO:** Aplicar filtro Gaussiano de tamaño $(2k+1) \times (2k+1)$ para obtener $I_{suave}$.  
   $$I_{suave} = I * G_{\sigma}$$

2. **CÁLCULO DE GRADIENTES:** Aplicar máscaras de Sobel para obtener derivadas parciales $G_x$ y $G_y$.  
   Calcular magnitud $M$ y dirección $\alpha$ para cada píxel.

3. **SUPRESIÓN DE NO MÁXIMOS:** **PARA** cada píxel $(i, j)$ en $M$:  
   * Determinar dirección del gradiente cuantizada (0, 45, 90, 135).  
   * Comparar $M(i, j)$ con vecinos en dicha dirección.  
   * **SI** $M(i, j)$ no es el máximo local, $M_{thin}(i, j) = 0$.

4. **UMBRALIZACIÓN DOBLE:** Clasificar píxeles en $M_{thin}$ como **Fuertes**, **Débiles** o **Suprimidos** según $T_{low}$ y $T_{high}$.

5. **SEGUIMIENTO DE BORDES (HISTERESIS):** **PARA** cada píxel débil:  
   * **SI** está conectado a un píxel fuerte (vecindad de 8), marcar como **Borde**.  
   * **SINO**, marcar como **Fondo**.

6. **RETORNAR** Imagen binaria final $E$.

## 7. Interpretación Geométrica y Comparativa

### 7.1 La Geometría de la Cresta
Desde una perspectiva geométrica, mientras que Sobel identifica una "zona de cambio" (una rampa de intensidad), Canny busca la **línea de cresta** exacta de esa rampa. La supresión de no máximos actúa como una operación de proyección que colapsa la pendiente del gradiente en un solo punto geométrico de máxima curvatura.

### 7.2 Comparativa: Sobel vs. Canny

| Característica | Operador de Sobel | Detector de Canny |
| :--- | :--- | :--- |
| **Complejidad** | Baja (1 solo paso) | Alta (múltiples etapas) |
| **Grosor del Borde** | Grueso (varios píxeles) | Delgado (exactamente 1 píxel) |
| **Sensibilidad al Ruido** | Alta | Baja (gracias al suavizado previo) |
| **Continuidad** | Bordes suelen presentar huecos | Bordes continuos (por histéresis) |
| **Precisión Matemática** | $O(h^2)$ en derivación | Óptima según criterios de SNR y Localización |

### 7.3 Conclusión
El método de Canny representa la evolución del sustento matemático de las **diferencias finitas**. No se limita a calcular la derivada parcial, sino que aplica una cadena de filtros numéricos (Gauss, Sobel, Supresión y Conectividad) para transformar una señal ruidosa en una representación geométrica precisa de los bordes.

# A-Frame: Framework de Realidad Virtual y Gráficos 3D

## 1. Sustento Tecnológico y el Grafo de Escena

A-Frame es un framework web de código abierto basado en la arquitectura **Entity-Component-System (ECS)** que se ejecuta sobre WebGL. Desde la perspectiva de los métodos numéricos, A-Frame es un motor que gestiona un **Grafo de Escena**, donde cada nodo representa una transformación espacial.

### 1.1 El Grafo de Escena como Estructura de Datos
El sistema organiza los objetos en una jerarquía de árbol. La posición, rotación y escala de cualquier entidad se calculan mediante la acumulación de transformaciones de sus ancestros.

### 1.2 Justificación Matemática: Transformaciones Homogéneas
Cualquier entidad $E$ en el espacio 3D se representa mediante una **Matriz de Transformación Homogénea $4 \times 4$**. Esto permite unificar traslación, rotación y escala en una sola operación matricial:

$$M = \begin{bmatrix} 
r_{11} & r_{12} & r_{13} & t_x \\ 
r_{21} & r_{22} & r_{23} & t_y \\ 
r_{31} & r_{32} & r_{33} & t_z \\ 
0 & 0 & 0 & 1 
\end{bmatrix}$$

Donde:
* El subbloque $3 \times 3$ superior izquierdo gestiona la **Rotación ($R$)** y la **Escala ($S$)**.
* La columna $[t_x, t_y, t_z]^T$ gestiona la **Traslación ($T$)**.

La posición final de un objeto "hijo" ($P_{final}$) respecto al origen global se obtiene mediante la multiplicación de matrices de su jerarquía:
$$P_{final} = M_{padre} \cdot M_{hijo} \cdot P_{local}$$

### 1.3 Coordenadas y Unidades
A-Frame utiliza un sistema de mano derecha (Right-handed coordinate system):
* **Eje X:** Positivo hacia la derecha.
* **Eje Y:** Positivo hacia arriba.
* **Eje Z:** Positivo hacia afuera de la pantalla (hacia el usuario).

> **Nota de Aplicación:** En VR, la unidad de medida está estandarizada como $1 \text{ unidad} = 1 \text{ metro}$, lo cual es crucial para la precisión del cálculo de la cámara y la percepción de profundidad del usuario.

## 2. Geometría Analítica y Álgebra Vectorial

En A-Frame, cada objeto y cada interacción se define mediante vectores en $\mathbb{R}^3$. La manipulación de estos vectores permite resolver problemas de posicionamiento, orientación y detección de colisiones.

### 2.1 Representación Vectorial
Cualquier punto o dirección en el espacio se define por un vector columna:
$$\vec{v} = \begin{bmatrix} x \\ y \\ z \end{bmatrix}$$

Para garantizar cálculos precisos (como en el caso de las direcciones de las luces o la vista de la cámara), es necesario trabajar con **Vectores Unitarios (Normalización)**:
$$\hat{u} = \frac{\vec{v}}{\|\vec{v}\|} = \frac{1}{\sqrt{x^2 + y^2 + z^2}} \begin{bmatrix} x \\ y \\ z \end{bmatrix}$$

### 2.2 Operaciones Fundamentales y su Aplicación Numérica

#### A. Producto Punto (Dot Product)
Se utiliza para calcular el ángulo entre dos vectores (como la mirada del usuario y un objeto) y en modelos de iluminación:
$$\vec{a} \cdot \vec{b} = \|\vec{a}\| \|\vec{b}\| \cos(\theta)$$
* **Uso:** Si $\vec{a} \cdot \vec{b} > 0$, el objeto está frente a la cámara; si es $< 0$, está detrás.

#### B. Producto Cruz (Cross Product)
Es esencial para calcular la **Normal de una Superficie** ($\vec{N}$), lo cual define cómo se renderiza una cara 3D:
$$\vec{N} = \vec{v_1} \times \vec{v_2} = \begin{vmatrix} \mathbf{i} & \mathbf{j} & \mathbf{k} \\ x_1 & y_1 & z_1 \\ x_2 & y_2 & z_2 \end{vmatrix}$$
* **Justificación:** El vector resultante es perpendicular al plano formado por $\vec{v_1}$ y $\vec{v_2}$, permitiendo al motor calcular el rebote de la luz.

### 2.3 Interpolación Lineal (LERP)
Para animaciones fluidas entre dos puntos $\vec{P_0}$ y $\vec{P_1}$, A-Frame utiliza interpolación numérica:
$$\vec{P}(t) = (1 - t)\vec{P_0} + t\vec{P_1}$$
Donde $t \in [0, 1]$ representa el paso del tiempo normalizado. Esta es la base de los métodos de aproximación para trayectorias de movimiento.

## 3. Rotaciones y Representación Algebraica: Cuaterniones

En el espacio 3D de A-Frame, las rotaciones se pueden definir mediante ángulos de Euler (Giro en X, Y, Z), pero internamente el motor utiliza **Cuaterniones** para realizar los cálculos numéricos de forma eficiente y evitar el fenómeno del *Gimbal Lock* (bloqueo de rotación).

### 3.1 El Problema de los Ángulos de Euler
Cuando rotamos un objeto usando tres ángulos ($\phi, \theta, \psi$), la matriz de rotación total es el producto de tres matrices individuales:
$$R_{total} = R_z(\psi)R_y(\theta)R_x(\phi)$$
**Justificación Numérica:** Este método pierde un grado de libertad cuando dos ejes de rotación se alinean (singularidad), lo que impide cálculos de trayectoria suaves en sistemas de navegación VR.

### 3.2 Solución mediante Cuaterniones ($\mathbb{H}$)
Un cuaternión es un número complejo en cuatro dimensiones que representa una rotación en torno a un eje arbitrario $\vec{u}$ por un ángulo $\alpha$:
$$\mathbf{q} = (s, \vec{v}) = \cos\left(\frac{\alpha}{2}\right) + \sin\left(\frac{\alpha}{2}\right)(u_x\mathbf{i} + u_y\mathbf{j} + u_z\mathbf{k})$$

Donde se cumple la identidad fundamental:
$$\mathbf{i}^2 = \mathbf{j}^2 = \mathbf{k}^2 = \mathbf{ijk} = -1$$

### 3.3 Esferical Linear Interpolation (SLERP)
Para animar rotaciones de manera numéricamente estable y uniforme, A-Frame utiliza la fórmula de **SLERP**, que interpola entre dos cuaterniones $q_0$ y $q_1$:

$$SLERP(q_0, q_1, t) = \frac{\sin((1-t)\Omega)}{\sin\Omega}q_0 + \frac{\sin(t\Omega)}{\sin\Omega}q_1$$

Donde:
* $t \in [0, 1]$ es el factor de interpolación.
* $\cos\Omega = q_0 \cdot q_1$ (producto escalar de los cuaterniones).

### 3.4 Ventajas en el Cómputo Numérico
1. **Menor costo computacional:** Multiplicar dos cuaterniones requiere menos operaciones de punto flotante que multiplicar dos matrices $3 \times 3$.
2. **Normalización:** Para que un cuaternión represente una rotación válida, debe ser un cuaternión unitario:
$$\|\mathbf{q}\| = \sqrt{s^2 + x^2 + y^2 + z^2} = 1$$
Esto permite corregir errores de redondeo numérico acumulados simplemente renormalizando el vector resultante.

## 4. El Pipeline de Renderizado: De 3D a la Pantalla 2D

Para que A-Frame pueda mostrar una escena en un monitor o en un visor de VR, debe resolver una serie de ecuaciones que proyectan los puntos del espacio euclidiano $\mathbb{R}^3$ al plano de la pantalla $\mathbb{R}^2$.

### 4.1 Matriz de Vista (View Matrix)
Primero, se transforma todo el universo de la escena para que la cámara sea el origen $(0,0,0)$. Si la cámara tiene una posición $C$ y una orientación $R$, la matriz de vista $V$ es la inversa de la matriz de transformación de la cámara:
$$V = (M_{camera})^{-1}$$
Esto justifica que mover la cámara hacia adelante es numéricamente equivalente a mover todo el mundo hacia atrás.

### 4.2 Matriz de Proyección de Perspectiva
Para simular el ojo humano, los objetos lejanos deben verse más pequeños. Esto se logra dividiendo las coordenadas por su profundidad ($z$). La matriz de proyección $P$ transforma un volumen truncado (frustum) en un cubo normalizado:

$$P = \begin{bmatrix} 
\frac{1}{aspect \cdot \tan(\frac{fov}{2})} & 0 & 0 & 0 \\ 
0 & \frac{1}{\tan(\frac{fov}{2})} & 0 & 0 \\ 
0 & 0 & -\frac{f+n}{f-n} & -\frac{2fn}{f-n} \\ 
0 & 0 & -1 & 0 
\end{bmatrix}$$

Donde:
* **$fov$:** Campo de visión (Field of View).
* **$aspect$:** Relación de aspecto de la pantalla (ancho/alto).
* **$n, f$:** Planos de corte cercano (near) y lejano (far).

### 4.3 Coordenadas de Dispositivo Normalizadas (NDC)
Tras multiplicar un punto $P_{world}$ por las matrices de Vista y Proyección, obtenemos coordenadas en un espacio intermedio. El paso final es la **División de Perspectiva**:
$$x_{ndc} = \frac{x_{clip}}{w_{clip}}, \quad y_{ndc} = \frac{y_{clip}}{w_{clip}}, \quad z_{ndc} = \frac{z_{clip}}{w_{clip}}$$

Este proceso asegura que todos los puntos visibles queden dentro del rango $[-1, 1]$. Cualquier valor fuera de este rango es descartado numéricamente (**Clipping**), optimizando el uso de la GPU.

### 4.4 Resumen del Flujo de Datos
La posición final de un vértice en pantalla ($v_{screen}$) se calcula como:
$$v_{screen} = P \cdot V \cdot M \cdot v_{local}$$

> **Justificación Numérica:** Este encadenamiento de multiplicaciones de matrices permite procesar miles de vértices en paralelo mediante hardware especializado (GPU), resolviendo sistemas lineales masivos en milisegundos.

## 5. Modelos Numéricos de Iluminación y Sombreado (Shading)

El renderizado en A-Frame se basa en calcular la intensidad de color en cada píxel mediante modelos matemáticos que aproximan la interacción de la luz con las superficies.

### 5.1 El Modelo de Reflexión de Lambert
Para superficies opacas (mate), A-Frame utiliza la ley del coseno de Lambert. La intensidad de iluminación $I$ en un punto depende del ángulo entre el vector de la luz $\vec{L}$ y el vector normal de la superficie $\vec{N}$.

La aproximación numérica se resuelve mediante el **Producto Punto**:
$$I = L_c \cdot K_d \cdot \max(0, \hat{N} \cdot \hat{L})$$

Donde:
* $L_c$: Intensidad y color de la fuente de luz.
* $K_d$: Coeficiente de reflexión difusa del material.
* $\hat{N} \cdot \hat{L}$: Coseno del ángulo entre los vectores normalizados. Si el ángulo es $> 90^\circ$, el producto punto es negativo y la luz no incide (valor 0).

### 5.2 Modelo Especular (Phong/Blinn-Phong)
Para simular el brillo en materiales como metal o plástico, se añade un componente que depende de la posición del observador (la cámara) $\vec{V}$. Se calcula un vector intermedio llamado **Vector Medio** ($\vec{H}$):

$$\vec{H} = \frac{\vec{L} + \vec{V}}{\|\vec{L} + \vec{V}\|}$$

La intensidad especular se aproxima como:
$$I_{spec} = K_s \cdot (\hat{N} \cdot \hat{H})^n$$
* $n$: Es el exponente de brillo (Shininess). A mayor $n$, el reflejo es más pequeño y definido.

### 5.3 Implementación en Shaders
Estos cálculos no se realizan en la CPU, sino que se delegan a la GPU mediante programas llamados **Shaders** escritos en GLSL. Existen dos aproximaciones principales:

| Método | Aplicación Numérica | Calidad Visual |
| :--- | :--- | :--- |
| **Gouraud Shading** | Calcula la iluminación en los vértices e interpola linealmente el color en el resto de la cara. | Baja (bordes visibles) |
| **Phong Shading** | Interpola las **normales** de los vértices y calcula la iluminación píxel por píxel. | Alta (suavidad matemática) |

### 5.4 Justificación del Error de Aproximación
Como A-Frame busca el rendimiento en tiempo real (60 FPS), utiliza estos modelos empíricos en lugar de resolver la **Ecuación de Renderizado de Kajiya** (que es una integral compleja). Esto es un ejemplo clásico de cómo los métodos numéricos sacrifican exactitud física por eficiencia computacional.

## 6. Arquitectura ECS y el Ciclo de Actualización Numérica

A-Frame utiliza el patrón **Entity-Component-System (ECS)**, el cual permite desacoplar los datos de la lógica, facilitando el procesamiento masivo de entidades.

### 6.1 Componentes y Sistemas
* **Entidades ($E$):** Identificadores únicos (puntos en el grafo).
* **Componentes ($C$):** Contenedores de datos (ej. posición, velocidad, masa).
* **Sistemas ($S$):** La lógica que opera sobre los datos. Desde el punto de vista numérico, un sistema es un **solucionador** que itera sobre un conjunto de estados.

### 6.2 El Método `tick`: Resolución en Tiempo Real
El motor de A-Frame ejecuta una función llamada `tick` aproximadamente cada $16.67$ ms (para alcanzar 60 FPS). Este es, en esencia, un paso de tiempo discreto $\Delta t$:

$$S_{t+1} = S_t + f(S_t, \Delta t)$$

Donde $f$ representa la función de actualización (física, animaciones, IA). Al ser un método iterativo, si $\Delta t$ varía (debido a caídas de rendimiento), el sistema debe compensarlo para mantener la consistencia temporal, una técnica común en la resolución de ecuaciones diferenciales ordinarias (EDO) para simulaciones físicas.

## 6. Arquitectura ECS y el Ciclo de Actualización Numérica

A-Frame utiliza el patrón **Entity-Component-System (ECS)**, el cual permite desacoplar los datos de la lógica, facilitando el procesamiento masivo de entidades.

### 6.1 Componentes y Sistemas
* **Entidades ($E$):** Identificadores únicos (puntos en el grafo).
* **Componentes ($C$):** Contenedores de datos (ej. posición, velocidad, masa).
* **Sistemas ($S$):** La lógica que opera sobre los datos. Desde el punto de vista numérico, un sistema es un **solucionador** que itera sobre un conjunto de estados.

### 6.2 El Método `tick`: Resolución en Tiempo Real
El motor de A-Frame ejecuta una función llamada `tick` aproximadamente cada $16.67$ ms (para alcanzar 60 FPS). Este es, en esencia, un paso de tiempo discreto $\Delta t$:

$$S_{t+1} = S_t + f(S_t, \Delta t)$$

Donde $f$ representa la función de actualización (física, animaciones, IA). Al ser un método iterativo, si $\Delta t$ varía (debido a caídas de rendimiento), el sistema debe compensarlo para mantener la consistencia temporal, una técnica común en la resolución de ecuaciones diferenciales ordinarias (EDO) para simulaciones físicas.

## 7. Implementación de una Escena Base

A continuación, se presenta la estructura declarativa de una escena. Aunque el usuario escribe etiquetas tipo HTML, el motor traduce cada etiqueta a los objetos matemáticos (matrices, mallas y vectores) descritos anteriormente.

```html
<a-scene>
  <a-sky color="#ECECEC"></a-sky>

  <a-box position="-1 0.5 -3" 
         rotation="0 45 0" 
         color="#4CC3D9"
         shadow>
  </a-box>

  <a-plane position="0 0 -4" 
           rotation="-90 0 0" 
           width="4" 
           height="4" 
           color="#7BC8A4">
  </a-plane>

  <a-camera position="0 1.6 0"></a-camera>
</a-scene>

### 7.1 Resumen de la Justificación
El uso de **A-Frame** en el contexto de **Métodos Numéricos** permite:

* **Visualización de Transformaciones:** Observar en tiempo real el efecto de las multiplicaciones matriciales y la composición de transformaciones lineales.
* **Simulación de Modelos Físicos:** Implementar algoritmos de colisión, gravedad y movimiento basados en **integración numérica** (como el método de Euler o Verlet).
* **Abstracción de WebGL:** Manipular el pipeline de renderizado (shading, proyección y rasterización) sin la necesidad de escribir manualmente miles de líneas de código de bajo nivel para la GPU, permitiendo enfocarse en la lógica matemática del espacio tridimensional.

---

> **Conclusión del Módulo:** Tanto los operadores de gradiente (**Sobel**), el algoritmo de optimización de bordes (**Canny**), como los motores de renderizado (**A-Frame**), comparten una base común: la discretización de funciones continuas para su resolución mediante cómputo digital. Mientras los primeros analizan la señal, el último la sintetiza, pero ambos dependen del rigor del álgebra lineal y el cálculo diferencial.

In [ ]:
from IPython.display import display, HTML
import os

# PEGUE AQUÍ TU CÓDIGO CON PEQUEÑAS MEJORAS DE FUNCIONALIDAD
# (Agregamos autoStart: false y un botón de inicio para evitar bloqueos)
html_content = """
<!DOCTYPE html>
<html lang="es">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>Filtros AR Profesionales</title>

  <script src="https://aframe.io/releases/1.4.2/aframe.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/mind-ar@1.2.5/dist/mindar-face-aframe.prod.js"></script>

  <style>
    body { margin: 0; overflow: hidden; background: black; font-family: sans-serif; }
    
    /* PANTALLA DE INICIO (Necesaria para permisos) */
    #start-screen {
        position: fixed; top: 0; left: 0; width: 100%; height: 100%;
        background: black; z-index: 9999;
        display: flex; flex-direction: column; align-items: center; justify-content: center;
        color: white;
    }
    #start-btn {
        background: #00e676; color: black; padding: 15px 30px;
        border: none; border-radius: 30px; font-size: 20px; font-weight: bold;
        cursor: pointer; margin-top: 20px;
    }

    /* TUS ESTILOS ORIGINALES */
    #ui {
      position: fixed;
      bottom: 20px;
      left: 50%;
      transform: translateX(-50%);
      display: none; /* Oculto al principio */
      gap: 10px;
      z-index: 10;
    }
    .control-btn {
      padding: 10px 14px;
      font-size: 20px;
      border-radius: 12px;
      border: 2px solid white;
      background: rgba(0,0,0,0.5);
      color: white;
      cursor: pointer;
    }
    .control-btn.active { background: #00e676; color: black; border-color: #00e676; }
  </style>
</head>

<body>

<div id="start-screen">
    <h1>Filtros AR</h1>
    <button id="start-btn" onclick="startAR()">📸 ACTIVAR CÁMARA</button>
</div>

<div id="ui">
  <button class="control-btn" onclick="toggle('sombrero', this)">🎩</button>
  <button class="control-btn" onclick="toggle('lentes', this)">🕶️</button>
  <button class="control-btn" onclick="toggle('bigote', this)">🧔</button>
  <button class="control-btn" onclick="toggle('lunar', this)">⚫</button>
</div>

<a-scene
  mindar-face="autoStart: false"
  embedded
  vr-mode-ui="enabled:false"
  device-orientation-permission-ui="enabled:false"
  renderer="colorManagement:true">

  <a-camera active="false" position="0 0 0"></a-camera>

  <a-entity mindar-face-target="anchorIndex:1">

    <a-entity id="sombrero" visible="false" position="0 0.9 0">
      <a-cylinder radius="1.1" height="0.05" color="#111"></a-cylinder>
      <a-cylinder position="0 0.35 0" radius="0.45" height="0.7" color="#111"></a-cylinder>
    </a-entity>

    <a-entity id="lentes" visible="false" position="0 0.30 0.06">
      <a-torus position="-0.18 0 0" radius="0.12" radius-tubular="0.012" color="#C0C0C0"></a-torus>
      <a-circle position="-0.18 0 0.01" radius="0.105"
        material="color:#3fa9f5; opacity:0.45; transparent:true"></a-circle>

      <a-torus position="0.18 0 0" radius="0.12" radius-tubular="0.012" color="#C0C0C0"></a-torus>
      <a-circle position="0.18 0 0.01" radius="0.105"
        material="color:#3fa9f5; opacity:0.45; transparent:true"></a-circle>

      <a-box position="0 0 0" scale="0.10 0.02 0.02" color="#C0C0C0"></a-box>
    </a-entity>

  </a-entity>

  <a-entity mindar-face-target="anchorIndex:330">
    <a-sphere
      id="lunar"
      radius="0.03"
      color="black"
      visible="false">
    </a-sphere>
  </a-entity>

  <a-entity mindar-face-target="anchorIndex:164"> <a-box
      id="bigote"
      position="0 -0.1 0"
      scale="0.5 0.1 0.1"
      color="black"
      visible="false">
    </a-box>
  </a-entity>

</a-scene>

<script>
    const startScreen = document.getElementById('start-screen');
    const startBtn = document.getElementById('start-btn');
    const ui = document.getElementById('ui');
    const scene = document.querySelector('a-scene');

    // 1. FUNCION DE INICIO (Para permisos de navegador)
    function startAR() {
        startBtn.innerText = "Cargando...";
        startBtn.disabled = true;
        
        // Arrancar MindAR
        scene.systems['mindar-face-system'].start();
    }

    // 2. EVENTO: CÁMARA LISTA
    scene.addEventListener('arReady', () => {
        startScreen.style.display = 'none'; // Quitar pantalla inicio
        ui.style.display = 'flex';          // Mostrar botones
    });

    // 3. TOGGLE CORREGIDO
    // El getAttribute devuelve string "true"/"false", hay que compararlo
    function toggle(id, btn) {
        const el = document.getElementById(id);
        const isVisible = el.getAttribute('visible');
        
        // Si es "true" o true, lo volvemos false. Si no, true.
        const newState = !(isVisible === true || isVisible === 'true');
        
        el.setAttribute('visible', newState);
        
        // Estilo visual del botón
        if(newState) btn.classList.add('active');
        else btn.classList.remove('active');
    }
</script>

</body>
</html>
"""

# GUARDAR EL ARCHIVO HTML
nombre_archivo = "mis_filtros.html"
with open(nombre_archivo, "w") as f:
    f.write(html_content)

print(f"✅ Archivo creado: {nombre_archivo}")

# BOTÓN PARA ABRIRLO
display(HTML(f"""
    <div style="background: #222; padding: 20px; border-radius: 10px; text-align: center;">
        <h2 style="color: white;">¡Listo!</h2>
        <a href="{nombre_archivo}" target="_blank" style="
            background: #00e676; color: black; text-decoration: none;
            padding: 15px 30px; font-size: 20px; font-weight: bold;
            border-radius: 30px; display: inline-block; margin-top: 10px;">
            🚀 ABRIR FILTROS EN PESTAÑA NUEVA
        </a>
    </div>
"""))

# Three.js: Motor de Gráficos Imperativo y Álgebra Lineal

## 1. Fundamentos Matemáticos y la Tubería de Datos (Pipeline)

A diferencia de los frameworks declarativos, **Three.js** es una biblioteca de bajo nivel que permite la manipulación directa de los búferes de memoria y las operaciones matriciales. Su funcionamiento se basa en la resolución de transformaciones en espacios de $n$-dimensiones.

### 1.1 El Objeto `BufferGeometry`: Eficiencia en Memoria
Desde la perspectiva numérica, Three.js no trata a los objetos como entidades abstractas, sino como **arreglos de datos tipados (TypedArrays)**. Una malla (mesh) se define por un conjunto de atributos vectoriales almacenados de forma contigua para maximizar el *throughput* de la GPU:

* **Vértices ($V$):** Un vector plano $[x_1, y_1, z_1, ..., x_n, y_n, z_n] \in \mathbb{R}^{3n}$.
* **Normales ($N$):** Vectores unitarios para el cálculo de iluminación.
* **Índices ($I$):** Define la topología de la malla mediante la conectividad de los vértices para formar triángulos.

### 1.2 La Cadena de Transformaciones (Modelo-Vista-Proyección)
Para renderizar un solo punto, Three.js debe resolver la ecuación fundamental de la computación gráfica, que es una cadena de multiplicaciones de matrices de $4 \times 4$:

$$P_{clip} = M_{proj} \cdot M_{view} \cdot M_{model} \cdot v_{local}$$

Cada componente tiene una justificación numérica específica:
1. **$M_{model}$ (World Matrix):** Transforma coordenadas locales al espacio global mediante traslación, rotación y escala.
2. **$M_{view}$ (Camera Matrix):** Es la matriz inversa de la posición de la cámara ($C^{-1}$), que reorienta el universo respecto al observador.
3. **$M_{proj}$ (Projection Matrix):** Aplica la distorsión de perspectiva, mapeando un volumen arbitrario a coordenadas normalizadas $[-1, 1]$.

### 1.3 El Grafo de Escena y Matrices Globales
Three.js utiliza un sistema de actualización recursiva para las matrices. Para cualquier objeto $i$, su matriz global $M_{global}^i$ se define como:

$$M_{global}^i = M_{global}^{padre} \cdot M_{local}^i$$

> **Dato para Métodos Numéricos:** Para optimizar el rendimiento, Three.js no recalcula estas matrices en cada frame a menos que la propiedad `.matrixWorldNeedsUpdate` sea verdadera, lo que ejemplifica una estrategia de **ahorro computacional** en sistemas dinámicos.

## 2. Geometría Computacional: Raycasting e Intersección

El **Raycasting** es la técnica que utiliza Three.js para resolver problemas de interactividad y detección de colisiones. Matemáticamente, consiste en proyectar una línea recta en el espacio y encontrar el punto de intersección con la geometría más cercana.

### 2.1 Ecuación Paramétrica del Rayo
Un rayo se define como una función lineal que depende de un parámetro de tiempo o distancia $t$:

$$\vec{R}(t) = \vec{O} + t\vec{D}$$

Donde:
* $\vec{O}$: Es el origen del rayo (usualmente la posición de la cámara).
* $\vec{D}$: Es el vector de dirección unitario ($\|\vec{D}\| = 1$).
* $t$: Es la distancia desde el origen ($t \geq 0$).

### 2.2 Algoritmo de Intersección Rayo-Triángulo (Möller-Trumbore)
Para determinar si un rayo golpea un objeto, Three.js debe resolver si el rayo intersecta alguno de los triángulos de la malla. Dado un triángulo con vértices $V_1, V_2, V_3$, un punto $P$ dentro del triángulo se define por sus **Coordenadas Baricéntricas** $(u, v)$:

$$P(u, v) = (1 - u - v)V_1 + uV_2 + vV_3$$

Igualando la ecuación del rayo con la del plano del triángulo, el sistema de ecuaciones lineales resultante es:

$$\vec{O} + t\vec{D} = (1 - u - v)V_1 + uV_2 + vV_3$$

Este sistema se resuelve numéricamente para $t, u, v$ mediante la regla de Cramer, lo que permite a la GPU o CPU determinar:
1. **Si hay intersección:** Si $u \geq 0, v \geq 0$ y $u + v \leq 1$.
2. **La distancia exacta:** El valor de $t$.

### 2.3 Frustum Culling: Optimización por Descarte
Antes de realizar los cálculos costosos de intersección, Three.js aplica métodos de **delimitación volumétrica**. Se envuelve cada objeto en una **Esfera de Delimitación (Bounding Sphere)** mínima:

$$\|\vec{P} - \vec{C}\|^2 \leq r^2$$

**Justificación Numérica:** Es mucho más barato computacionalmente verificar la intersección con una esfera (una sola ecuación cuadrática) que con miles de triángulos. Si el rayo no toca la esfera, se descarta el objeto inmediatamente, reduciendo la complejidad algorítmica de $O(n)$ a $O(1)$ para ese objeto.

## 3. Interpolación y Curvas Paramétricas

En Three.js, la animación y el movimiento de cámaras u objetos se basan en la aproximación de trayectorias mediante funciones polinómicas. Esto es fundamental para evitar movimientos "robóticos" y asegurar la continuidad en las derivadas de la posición (velocidad y aceleración).

### 3.1 Curvas de Bézier (Interpolación Polinómica)
Para definir trayectorias suaves, se utilizan las **Curvas de Bézier**, que se basan en los **Polinomios de Bernstein**. Una curva de tercer grado (cúbica) se define mediante cuatro puntos de control $P_0, P_1, P_2, P_3$:

$$B(t) = (1-t)^3 P_0 + 3(1-t)^2 t P_1 + 3(1-t) t^2 P_2 + t^3 P_3, \quad t \in [0, 1]$$

**Justificación Numérica:** Esta fórmula garantiza que la curva sea tangente a los segmentos de control, permitiendo un control geométrico preciso de la trayectoria sin necesidad de resolver sistemas de ecuaciones globales pesados.

### 3.2 Splines de Catmull-Rom
A diferencia de Bézier, donde la curva no siempre pasa por los puntos de control, Three.js utiliza frecuentemente **Catmull-Rom Splines** para que la trayectoria pase exactamente por los nodos definidos. La ecuación local para un segmento entre $P_i$ y $P_{i+1}$ es:

$$P(t) = \frac{1}{2} \begin{bmatrix} 1 & t & t^2 & t^3 \end{bmatrix} \begin{bmatrix} 0 & 2 & 0 & 0 \\ -1 & 0 & 1 & 0 \\ 2 & -5 & 4 & -1 \\ -1 & 3 & -3 & 1 \end{bmatrix} \begin{bmatrix} P_{i-1} \\ P_i \\ P_{i+1} \\ P_{i+2} \end{bmatrix}$$

* **Continuidad $C^1$:** Este método asegura que la primera derivada (velocidad) sea continua en todos los puntos, eliminando cambios bruscos de dirección.

### 3.3 El Ciclo de Animación y Delta Time ($\Delta t$)
En métodos numéricos, la integración del movimiento requiere un paso de tiempo. Three.js utiliza el reloj del sistema para calcular el `delta`:

$$\text{Posición}_{nuevo} = \text{Posición}_{anterior} + (\text{Velocidad} \cdot \Delta t)$$

Si la velocidad no es constante, se aplican métodos de integración como **Euler Simple** o, en simulaciones más avanzadas dentro de Three.js, **Integración de Verlet**, que es más estable numéricamente para sistemas de partículas:

$$x_{t+ \Delta t} = 2x_t - x_{t- \Delta t} + a \cdot \Delta t^2$$

## 4. Iluminación Avanzada y Mapeo de Normales (Normal Mapping)

En Three.js, la apariencia de una superficie no depende solo de la cantidad de triángulos, sino de cómo se manipulan los vectores normales en el nivel de los fragmentos (píxeles). Esto se basa en la aplicación de **Álgebra Lineal** para simular micro-geometría.

### 4.1 Espacio Tangente y Base TBN
Para aplicar un mapa de normales (una textura que indica hacia dónde "mira" cada píxel), Three.js debe construir un sistema de coordenadas local para cada punto de la superficie, llamado **Espacio Tangente**. Este se define por tres vectores ortonormales:
* **$\vec{T}$ (Tangente):** Vector en la dirección del eje U de la textura.
* **$\vec{B}$ (Bitangente):** Vector en la dirección del eje V de la textura.
* **$\vec{N}$ (Normal):** Vector perpendicular a la cara.

Estos vectores forman la **Matriz TBN**, que permite transformar vectores del "Espacio de Textura" al "Espacio del Mundo":
$$M_{TBN} = \begin{bmatrix} T_x & B_x & N_x \\ T_y & B_y & N_y \\ T_z & B_z & N_z \end{bmatrix}$$

### 4.2 Perturbación de la Normal
El valor de color de un píxel en el mapa de normales $(R, G, B)$ se decodifica para obtener un vector unitario en el rango $[-1, 1]$:
$$\vec{n}_{local} = \begin{bmatrix} 2R - 1 \\ 2G - 1 \\ 2B - 1 \end{bmatrix}$$

La normal final utilizada para el cálculo de iluminación ($\vec{n}_{final}$) es el producto de la matriz TBN por este vector local:
$$\vec{n}_{final} = M_{TBN} \cdot \vec{n}_{local}$$

### 4.3 El Modelo de PBR (Physically Based Rendering)
Three.js utiliza en su `MeshStandardMaterial` un modelo numérico avanzado llamado **Ecuación de Reflectancia**, que es una integral de iluminación:
$$L_o(p, \omega_o) = \int_{\Omega} f_r(p, \omega_i, \omega_o) L_i(p, \omega_i) (\vec{n} \cdot \omega_i) d\omega_i$$

**Justificación Numérica:** Como resolver esta integral en tiempo real es imposible, Three.js utiliza una aproximación de **Suma de Riemann** o **Importancia de Muestreo (Importance Sampling)** para simular cómo la luz se refleja en micro-facetas, basándose en la función de distribución de micro-facetas de **Trowbridge-Reitz (GGX)**.

### 4.4 Cálculo de Sombras (Shadow Mapping)
Las sombras se calculan mediante una comparación de profundidad ($z$). Para cada píxel, el motor resuelve:
$$Visible = \begin{cases} 1 & \text{si } z_{píxel} \leq z_{shadow\_map} + \epsilon \\ 0 & \text{en otro caso} \end{cases}$$
*Donde $\epsilon$ es un factor de sesgo (bias) introducido para corregir errores de precisión numérica de punto flotante.*

## 5. El Ciclo de Renderizado (Render Loop) y Estabilidad Numérica

En Three.js, la generación de imágenes no es un evento estático, sino un proceso iterativo continuo que debe mantener la estabilidad numérica para evitar errores de precisión o "saltos" en la simulación.

### 5.1 Discretización del Tiempo
Cada iteración del ciclo de renderizado representa un paso de tiempo discreto $\Delta t$. Para que las simulaciones físicas sean consistentes, Three.js utiliza el método de `requestAnimationFrame`, que busca sincronizarse con la tasa de refresco del monitor.

La actualización del estado del sistema se define como:
$$X_{t+1} = X_t + \int_{t}^{t+\Delta t} v(t) dt \approx X_t + v_t \cdot \Delta t$$

### 5.2 Manejo del Error de Redondeo (Floating Point Precision)
Dado que WebGL utiliza precisión de punto flotante de 32 bits, en escenas con escalas masivas (ej. simulaciones astronómicas) se presenta el error de **Z-Fighting**.
Este fenómeno ocurre cuando dos profundidades $z_1$ y $z_2$ son tan cercanas que la precisión numérica no puede distinguirlas:
$$|z_1 - z_2| < \epsilon$$
Donde $\epsilon$ es la resolución del búfer de profundidad. Three.js permite mitigar esto mediante el uso de un **Logarithmic Depth Buffer**, que redistribuye la precisión de forma no lineal (logarítmica) para dar más detalle a objetos lejanos.

## 5. El Ciclo de Renderizado (Render Loop) y Estabilidad Numérica

En Three.js, la generación de imágenes no es un evento estático, sino un proceso iterativo continuo que debe mantener la estabilidad numérica para evitar errores de precisión o "saltos" en la simulación.

### 5.1 Discretización del Tiempo
Cada iteración del ciclo de renderizado representa un paso de tiempo discreto $\Delta t$. Para que las simulaciones físicas sean consistentes, Three.js utiliza el método de `requestAnimationFrame`, que busca sincronizarse con la tasa de refresco del monitor.

La actualización del estado del sistema se define como:
$$X_{t+1} = X_t + \int_{t}^{t+\Delta t} v(t) dt \approx X_t + v_t \cdot \Delta t$$

### 5.2 Manejo del Error de Redondeo (Floating Point Precision)
Dado que WebGL utiliza precisión de punto flotante de 32 bits, en escenas con escalas masivas (ej. simulaciones astronómicas) se presenta el error de **Z-Fighting**.
Este fenómeno ocurre cuando dos profundidades $z_1$ y $z_2$ son tan cercanas que la precisión numérica no puede distinguirlas:
$$|z_1 - z_2| < \epsilon$$
Donde $\epsilon$ es la resolución del búfer de profundidad. Three.js permite mitigar esto mediante el uso de un **Logarithmic Depth Buffer**, que redistribuye la precisión de forma no lineal (logarítmica) para dar más detalle a objetos lejanos.

## 6. Estructura de Implementación y Código Base

A diferencia de A-Frame, en Three.js la configuración de la escena requiere la instanciación explícita de los objetos matemáticos (Cámara, Escena, Renderizador).

### 6.1 Componentes Esenciales del Script
El siguiente bloque muestra cómo se traduce la teoría de matrices y vectores a código funcional:

```javascript
// 1. Inicialización del Motor (Espacio de Renderizado)
const scene = new THREE.Scene();
const camera = new THREE.PerspectiveCamera(75, window.innerWidth / window.innerHeight, 0.1, 1000);
const renderer = new THREE.WebGLRenderer();

// 2. Creación de Geometría Discreta
// Se define la topología de la malla (8 vértices, 12 caras triangulares)
const geometry = new THREE.BoxGeometry(1, 1, 1);

// 3. Definición del Material (Modelo de Iluminación)
// Utiliza algoritmos de sombreado de fragmentos para calcular la luz
const material = new THREE.MeshStandardMaterial({ color: 0x00ff00 });

// 4. Creación de la Malla (Mesh = Geometría + Material)
const cube = new THREE.Mesh(geometry, material);
scene.add(cube);

// 5. Bucle de Animación (Método Iterativo)
function animate() {
  requestAnimationFrame(animate);

  // Aplicación de rotación incremental (Transformación de matriz local)
  cube.rotation.x += 0.01;
  cube.rotation.y += 0.01;

  // Resolución de la Matriz de Proyección y dibujado en el Canvas
  renderer.render(scene, camera);
}
animate();

In [ ]:
from IPython.display import display, HTML

codigo_seguro = """
<div id="app-container">
    <div class="controls">
        <button id="btn-glasses">👓 Lentes</button>
        <button id="btn-mustache">🥸 Bigote</button>
        <button id="btn-hat">🎩 Sombrero</button>
        <button id="btn-mole">🟤 Lunar</button>
        <button id="btn-points">🟢 Puntos (Debug)</button>
    </div>

    <div class="viewport">
        <video id="input-video" style="display:none" autoplay muted playsinline></video>
        <canvas id="output-canvas"></canvas>
        <div id="status-text">Iniciando sistema...</div>
    </div>
</div>

<script src="https://cdn.jsdelivr.net/npm/@mediapipe/face_mesh/face_mesh.js"></script>
<script src="https://cdn.jsdelivr.net/npm/@mediapipe/camera_utils/camera_utils.js"></script>

<style>
    #app-container {
        font-family: sans-serif;
        width: 640px;
        margin: 0 auto;
        background: #222;
        padding: 10px;
        border-radius: 8px;
        color: white;
    }

    .viewport {
        position: relative;
        width: 640px;
        height: 480px;
        background: black;
        border: 2px solid #444;
        overflow: hidden;
    }

    canvas {
        width: 100%;
        height: 100%;
        object-fit: cover;
    }

    .controls {
        display: flex;
        gap: 5px;
        margin-bottom: 10px;
        justify-content: center;
        flex-wrap: wrap;
    }

    button {
        background: #444;
        color: white;
        border: 1px solid #666;
        padding: 8px 12px;
        cursor: pointer;
        border-radius: 4px;
        font-weight: bold;
    }

    button:hover { background: #555; }
    
    /* Clase para botón activo (Verde) */
    button.active {
        background: #00e676;
        color: black;
        border-color: #00a152;
    }

    #status-text {
        position: absolute;
        bottom: 10px;
        left: 10px;
        background: rgba(0, 0, 0, 0.7);
        padding: 5px 10px;
        border-radius: 4px;
        font-size: 14px;
        pointer-events: none;
    }
</style>

<script>
(function() {
    // 1. Configuración Inicial
    const container = document.getElementById('app-container');
    const videoElement = container.querySelector('#input-video');
    const canvasElement = container.querySelector('#output-canvas');
    const ctx = canvasElement.getContext('2d');
    const statusText = container.querySelector('#status-text');
    
    // Forzamos tamaño
    canvasElement.width = 640;
    canvasElement.height = 480;

    // Estado de filtros
    const state = {
        glasses: false,
        mustache: false,
        hat: false,
        mole: false,
        points: false // Activado por defecto para probar
    };

    // 2. Conectar Botones
    function setupButton(id, key) {
        const btn = container.querySelector('#' + id);
        if(!btn) return;
        btn.onclick = () => {
            state[key] = !state[key];
            // Visualmente cambiar el botón
            if(state[key]) btn.classList.add('active');
            else btn.classList.remove('active');
            console.log("Toggle " + key + ": " + state[key]);
        };
    }

    setupButton('btn-glasses', 'glasses');
    setupButton('btn-mustache', 'mustache');
    setupButton('btn-hat', 'hat');
    setupButton('btn-mole', 'mole');
    setupButton('btn-points', 'points');

    // 3. Lógica de Dibujo (Filtros 2D)
    function drawScene(results) {
        // Limpiar
        ctx.clearRect(0, 0, canvasElement.width, canvasElement.height);
        
        // A. Dibujar la imagen de la cámara DE FONDO
        ctx.drawImage(results.image, 0, 0, canvasElement.width, canvasElement.height);

        // B. Verificar si hay cara
        if (results.multiFaceLandmarks && results.multiFaceLandmarks.length > 0) {
            statusText.innerText = "✅ Rostro detectado - Filtros activos";
            statusText.style.color = "#00e676";
            
            const landmarks = results.multiFaceLandmarks[0];
            drawFilters(landmarks);
        } else {
            statusText.innerText = "⚠️ Buscando rostro... (Acércate a la cámara)";
            statusText.style.color = "orange";
        }
    }

    function drawFilters(landmarks) {
        const w = canvasElement.width;
        const h = canvasElement.height;

        // Puntos Clave
        const leftEye = landmarks[33];
        const rightEye = landmarks[263];
        const noseTip = landmarks[1];
        const upperLip = landmarks[13];
        const forehead = landmarks[10];
        const leftCheek = landmarks[234];

        // Matemáticas para posición y tamaño
        const cx = (leftEye.x + rightEye.x) / 2 * w;
        const cy = (leftEye.y + rightEye.y) / 2 * h;
        const dx = (rightEye.x - leftEye.x) * w;
        const dy = (rightEye.y - leftEye.y) * h;
        const dist = Math.sqrt(dx*dx + dy*dy); // Distancia entre ojos
        const angle = Math.atan2(dy, dx); // Inclinación de la cabeza

        ctx.save(); // Guardar estado para rotaciones

        // --- 1. PUNTOS (DEBUG) ---
        if (state.points) {
            ctx.fillStyle = "#00FF00";
            for (let point of landmarks) {
                ctx.beginPath();
                ctx.arc(point.x * w, point.y * h, 1, 0, 2 * Math.PI);
                ctx.fill();
            }
        }

        // --- 2. LENTES ---
        if (state.glasses) {
            ctx.translate(cx, cy);
            ctx.rotate(angle);
            
            ctx.strokeStyle = "black";
            ctx.lineWidth = 5;
            // Marco Izq
            ctx.strokeRect(-dist*0.9, -dist*0.3, dist*0.8, dist*0.5);
            // Marco Der
            ctx.strokeRect(dist*0.1, -dist*0.3, dist*0.8, dist*0.5);
            // Puente
            ctx.beginPath();
            ctx.moveTo(-dist*0.1, 0);
            ctx.lineTo(dist*0.1, 0);
            ctx.stroke();

            ctx.rotate(-angle); // Deshacer rotación
            ctx.translate(-cx, -cy); // Deshacer traslación
        }

        // --- 3. BIGOTE ---
        if (state.mustache) {
            const mx = upperLip.x * w;
            const my = upperLip.y * h;
            
            ctx.translate(mx, my);
            ctx.rotate(angle);

            ctx.fillStyle = "#3e2723";
            ctx.beginPath();
            ctx.ellipse(0, -dist*0.1, dist*0.6, dist*0.15, 0, 0, Math.PI*2);
            ctx.fill();

            ctx.rotate(-angle);
            ctx.translate(-mx, -my);
        }

        // --- 4. SOMBRERO ---
        if (state.hat) {
            const hx = forehead.x * w;
            const hy = forehead.y * h;

            ctx.translate(hx, hy);
            ctx.rotate(angle);

            ctx.fillStyle = "#1a237e"; // Azul oscuro
            // Copa
            ctx.fillRect(-dist*0.8, -dist*1.5, dist*1.6, dist*1.0);
            // Ala
            ctx.fillRect(-dist*1.2, -dist*0.5, dist*2.4, dist*0.2);

            ctx.rotate(-angle);
            ctx.translate(-hx, -hy);
        }

        // --- 5. LUNAR ---
        if (state.mole) {
            const lx = leftCheek.x * w;
            const ly = leftCheek.y * h;
            ctx.fillStyle = "black";
            ctx.beginPath();
            ctx.arc(lx + dist*0.2, ly, dist*0.08, 0, Math.PI*2);
            ctx.fill();
        }

        ctx.restore();
    }

    // 4. Iniciar MediaPipe FaceMesh
    const faceMesh = new FaceMesh({
        locateFile: (file) => `https://cdn.jsdelivr.net/npm/@mediapipe/face_mesh/${file}`
    });

    faceMesh.setOptions({
        maxNumFaces: 1,
        refineLandmarks: true,
        minDetectionConfidence: 0.5,
        minTrackingConfidence: 0.5
    });

    faceMesh.onResults(drawScene);

    // 5. Iniciar Cámara
    const camera = new Camera(videoElement, {
        onFrame: async () => {
            await faceMesh.send({image: videoElement});
        },
        width: 640,
        height: 480
    });

    statusText.innerText = "Solicitando cámara...";
    camera.start()
        .then(() => statusText.innerText = "Cámara activa. Cargando modelo...")
        .catch(err => statusText.innerText = "Error de cámara: " + err);

})();
</script>
"""

display(HTML(codigo_seguro))

# MediaPipe: Inferencia Perceptual y Optimización de Grafos

## 1. Fundamentos de Visión Computacional y Redes Convolucionales

MediaPipe es un framework basado en el paradigma de **ML Pipelines**. Su función principal en el contexto numérico es la **regresión de coordenadas**, donde una imagen de entrada $I$ se procesa para obtener un conjunto de vectores de características.

### 1.1 Inferencia como Función de Mapeo
La detección de una cara o mano se puede expresar como una función $f$ que mapea el espacio de la imagen al espacio de parámetros:
$$f(I) = \Theta$$
Donde:
* $I \in \mathbb{R}^{W \times H \times 3}$: Es la matriz de píxeles (ancho, alto, canales RGB).
* $\Theta = \{\vec{P}_1, \vec{P}_2, ..., \vec{P}_n\}$: Es el conjunto de puntos clave (landmarks), donde cada $\vec{P}_i = (x, y, z)$.

### 1.2 La Convolución Discreta (Sustento del Modelo)
Para extraer características de la imagen, MediaPipe utiliza **Redes Neuronales Convolucionales (CNN)**. La operación fundamental es la convolución discreta entre la imagen $I$ y un filtro (kernel) $K$ de tamaño $m \times n$:

$$(I * K)(i, j) = \sum_{m} \sum_{n} I(i-m, j-n) \cdot K(m, n)$$

**Justificación Numérica:** Este proceso actúa como un detector de gradientes y patrones (similar a los filtros de Sobel/Canny pero con pesos aprendidos), reduciendo la dimensionalidad de la imagen a un vector de características abstractas.

### 1.3 El Grafo de Flujo (Calculators)
MediaPipe organiza el procesamiento en un **Grafo Acíclico Dirigido (DAG)**. Cada nodo es una "calculadora" que recibe un paquete de datos $T$ (Tensor) y emite un resultado tras aplicar una transformación numérica:
$$T_{out} = \phi(T_{in})$$
Esta arquitectura permite el **paralelismo masivo**, resolviendo múltiples etapas de la tubería de datos simultáneamente.

### 1.4 Normalización y Coordenadas Relativas
MediaPipe no entrega coordenadas en píxeles, sino valores normalizados en el rango $[0, 1]$. La conversión a coordenadas de pantalla (necesaria para Three.js) es una **Transformación Lineal Escalar**:

$$P_{pixel} = \begin{bmatrix} x_{norm} \cdot W \\ y_{norm} \cdot H \\ z_{norm} \cdot W \end{bmatrix}$$

*Donde $W$ y $H$ son el ancho y alto de la imagen.* La coordenada $z$ se calcula de manera relativa al centro de gravedad del objeto detectado, permitiendo una estimación de profundidad sin necesidad de una cámara estéreo.

## 2. El Modelo Face Mesh y Regresión de Puntos Clave

El sistema **Face Mesh** de MediaPipe predice la posición de 468 puntos (landmarks) en 3D. A diferencia de un detector de objetos simple que solo encuentra una caja delimitadora, este modelo realiza una **Regresión de Coordenadas de Alta Dimensionalidad**.

### 2.1 Función de Pérdida y Optimización
Para que el modelo aprenda a colocar los puntos correctamente, durante su entrenamiento se minimiza una **Función de Pérdida (Loss Function)**, generalmente basada en el **Error Cuadrático Medio (MSE)** entre la posición predicha ($\hat{P}$) y la real ($P$):

$$\mathcal{L} = \frac{1}{n} \sum_{i=1}^{n} \|\vec{P}_i - \hat{\vec{P}}_i\|^2$$

Donde $n = 468$. El modelo utiliza el **Gradiente Descendente** para ajustar los pesos de la red y minimizar este error numérico.

### 2.2 Reconstrucción de la Superficie (Topología Fija)
MediaPipe no solo entrega puntos aislados; los entrega con una **conectividad predefinida**. Esto significa que los 468 puntos forman una malla de triángulos fija.
* **Justificación:** Se utiliza una matriz de adyacencia constante para definir cómo se conectan los vértices. Esto permite calcular el vector normal ($\vec{N}$) de cualquier parte de la cara mediante el producto cruz de las aristas del triángulo formado por los landmarks:
$$\vec{N}_{cara} = (\vec{P}_{v2} - \vec{P}_{v1}) \times (\vec{P}_{v3} - \vec{P}_{v1})$$

### 2.3 Estimación de Profundidad Relativa
Una de las mayores proezas numéricas de MediaPipe es obtener la coordenada $z$ (profundidad) a partir de una cámara 2D. Esto lo logra mediante **Inferencia de Profundidad**:
1. El modelo asume un modelo de cabeza promedio (Weak Perspective Projection).
2. La coordenada $z$ de los puntos se predice basándose en la deformación y el tamaño relativo de las facciones.
3. El origen ($z=0$) se sitúa generalmente en el centro de masa de la cara o en el puente de la nariz.

### 2.4 Alineación Espacial: Transformaciones Afines
Para alinear los filtros (lentes, sombreros) con la cara detectada, se resuelven transformaciones afines que mantienen el paralelismo de las líneas. Si tenemos tres puntos de referencia en la cara (ojos y nariz), podemos calcular la **Matriz de Transformación Afín $A$** que mapea el objeto al espacio facial:

$$\begin{bmatrix} x' \\ y' \end{bmatrix} = \begin{bmatrix} a_{11} & a_{12} \\ a_{21} & a_{22} \end{bmatrix} \begin{bmatrix} x \\ y \end{bmatrix} + \begin{bmatrix} t_x \\ t_y \end{bmatrix}$$

Este sistema de ecuaciones se resuelve en cada frame para asegurar que el "filtro" siga el movimiento del usuario con precisión milimétrica.

## 3. El Problema PnP y Estimación de la Pose (Head Pose Estimation)

Para que los filtros 3D se orienten correctamente, MediaPipe debe resolver el problema de **Perspectiva de n-Puntos (PnP)**. Esto consiste en encontrar la rotación ($R$) y traslación ($\vec{t}$) de la cabeza a partir de las coordenadas 2D observadas en la imagen.

### 3.1 El Modelo de Cámara Pinhole
El sistema asume que los puntos del mundo 3D ($X, Y, Z$) se proyectan en el plano de la imagen ($u, v$) siguiendo la ecuación de proyección central:

$$s \begin{bmatrix} u \\ v \\ 1 \end{bmatrix} = \mathbf{K} [ \mathbf{R} | \vec{t} ] \begin{bmatrix} X \\ Y \\ Z \\ 1 \end{bmatrix}$$

Donde:
* **$\mathbf{K}$:** Es la matriz de parámetros intrínsecos de la cámara (focal, centro óptico).
* **$[\mathbf{R}|\vec{t}]$:** Es la matriz de parámetros extrínsecos (la pose que queremos hallar).
* **$s$:** Es un factor de escala.

### 3.2 Resolución mediante Iteración Numérica
Como la imagen es una proyección 2D de un objeto 3D, el problema está **sobredeterminado**. MediaPipe utiliza algoritmos como **Levenberg-Marquardt** para minimizar el **Error de Reproyección**:

$$\min_{\mathbf{R}, \vec{t}} \sum_{i=1}^{n} \| p_i - \text{proj}(\mathbf{K}, \mathbf{R}, \vec{t}, P_i) \|^2$$

Este es un método numérico de optimización que ajusta la rotación y posición hasta que los puntos del modelo 3D "calzan" perfectamente sobre los píxeles de la cara detectada.

### 3.3 Extracción de los Ángulos de Euler (Tait-Bryan)
Una vez obtenida la matriz de rotación $\mathbf{R}$, se descomponen los ángulos para aplicarlos en Three.js o A-Frame:
* **Pitch (X):** Inclinación arriba/abajo.
* **Yaw (Y):** Giro izquierda/derecha.
* **Roll (Z):** Inclinación lateral.

La matriz de rotación se define como el producto:
$$\mathbf{R} = R_z(\psi)R_y(\theta)R_x(\phi)$$

### 3.4 Estabilización de la Pose (Filtrado de Señal)
Debido al ruido en la captura de video, los valores de $R$ y $t$ pueden oscilar. Para evitar esto, se aplican técnicas de **Suavizado Exponencial**:

$$S_t = \alpha \cdot X_t + (1 - \alpha) \cdot S_{t-1}$$

Donde $\alpha \in [0, 1]$ es el factor de suavizado. Este método numérico permite que el sombrero o los lentes no "tiemblen" ante pequeñas variaciones de luz o movimiento del sensor.

## 4. Estabilización de Datos: El Filtro "One Euro" ($1€$)

En la visión computacional en tiempo real, los landmarks detectados suelen presentar un fenómeno de "jitter" (vibración aleatoria) debido a errores de cuantización en la imagen y ruido en el sensor. MediaPipe utiliza el **Filtro One Euro**, un filtro de paso bajo adaptativo de primer orden.

### 4.1 La Limitación de los Filtros Tradicionales
Un filtro de paso bajo estándar tiene un compromiso (trade-off) crítico:
1. **Si el corte es bajo:** Se elimina el ruido, pero se introduce **lag** (retraso) en movimientos rápidos.
2. **Si el corte es alto:** El sistema responde rápido, pero los puntos tiemblan cuando el usuario está quieto.

### 4.2 El Modelo Matemático Adaptativo
El filtro $1€$ resuelve esto ajustando dinámicamente su frecuencia de corte $f_c$ basándose en la velocidad de la señal. La ecuación fundamental del filtro de paso bajo es:

$$\hat{X}_k = \alpha X_k + (1 - \alpha) \hat{X}_{k-1}$$

Donde el factor de suavizado $\alpha$ se calcula a partir de la frecuencia de corte:
$$\alpha = \frac{1}{1 + \frac{1}{2\pi \cdot \Delta t \cdot f_c}}$$

### 4.3 Adaptación mediante la Velocidad (Derivada Discreta)
La clave numérica está en definir $f_c$ como una función de la velocidad de cambio de la señal ($\dot{X}$):

1. **Cálculo de la velocidad:** $\dot{X}_k = \frac{X_k - \hat{X}_{k-1}}{\Delta t}$
2. **Suavizado de la velocidad:** $\hat{\dot{X}}_k = \text{LowPass}(\dot{X}_k, \alpha_{speed})$
3. **Frecuencia de corte adaptativa:** $f_c = f_{c_{min}} + \beta |\hat{\dot{X}}_k|$

Donde:
* **$f_{c_{min}}$:** Evita el jitter cuando el usuario está quieto.
* **$\beta$:** Coeficiente de intensidad que aumenta la frecuencia de corte si hay mucha velocidad, reduciendo el lag.

### 4.4 Justificación en Métodos Numéricos
Este algoritmo representa un **controlador proporcional** simple aplicado al procesamiento de señales. Permite que MediaPipe entregue coordenadas que se sienten "físicas" y naturales, eliminando la alta frecuencia del ruido sin sacrificar la respuesta inmediata del sistema ante movimientos bruscos.

## 5. Procesamiento de Tensores y Aceleración en GPU

El motor de cálculo de MediaPipe no opera sobre píxeles individuales de forma secuencial, sino que utiliza **Álgebra Multilineal** para procesar la información mediante Tensores.

### 5.1 ¿Qué es un Tensor en MediaPipe?
Un tensor es una generalización de los vectores y matrices a $n$ dimensiones. En el caso del video, la entrada es un tensor de cuarto orden:
$$\mathbf{T} \in \mathbb{R}^{B \times H \times W \times C}$$
Donde:
* **$B$:** Batch size (usualmente 1 en tiempo real).
* **$H, W$:** Altura y ancho de la imagen.
* **$C$:** Canales de color (3 para RGB).

### 5.2 Cuantización y Optimización Numérica
Para ejecutar estos modelos en navegadores o móviles, MediaPipe utiliza **Cuantización**. Esto consiste en reducir la precisión de los pesos de la red neuronal de `float32` (32 bits) a `int8` (8 bits). 

**Justificación Numérica:** Aunque se introduce un pequeño error de redondeo, el volumen de datos se reduce en un **75%**, permitiendo que las operaciones de producto punto en las capas de la red se realicen mucho más rápido en la GPU.

### 5.3 Implementación vía WebGL/WebGPU
MediaPipe delega las multiplicaciones matriciales a la GPU mediante **Shaders de Computación**. Esto permite resolver el sistema de la red neuronal en paralelo:
$$\vec{y} = \sigma(\mathbf{W}\vec{x} + \vec{b})$$
Donde $\mathbf{W}$ es la matriz de pesos, $\vec{x}$ los datos de entrada, $\vec{b}$ el sesgo y $\sigma$ la función de activación (ej. ReLU). Millones de estas operaciones ocurren cada 16ms.

## 5. Procesamiento de Tensores y Aceleración en GPU

El motor de cálculo de MediaPipe no opera sobre píxeles individuales de forma secuencial, sino que utiliza **Álgebra Multilineal** para procesar la información mediante Tensores.

### 5.1 ¿Qué es un Tensor en MediaPipe?
Un tensor es una generalización de los vectores y matrices a $n$ dimensiones. En el caso del video, la entrada es un tensor de cuarto orden:
$$\mathbf{T} \in \mathbb{R}^{B \times H \times W \times C}$$
Donde:
* **$B$:** Batch size (usualmente 1 en tiempo real).
* **$H, W$:** Altura y ancho de la imagen.
* **$C$:** Canales de color (3 para RGB).

### 5.2 Cuantización y Optimización Numérica
Para ejecutar estos modelos en navegadores o móviles, MediaPipe utiliza **Cuantización**. Esto consiste en reducir la precisión de los pesos de la red neuronal de `float32` (32 bits) a `int8` (8 bits). 

**Justificación Numérica:** Aunque se introduce un pequeño error de redondeo, el volumen de datos se reduce en un **75%**, permitiendo que las operaciones de producto punto en las capas de la red se realicen mucho más rápido en la GPU.

### 5.3 Implementación vía WebGL/WebGPU
MediaPipe delega las multiplicaciones matriciales a la GPU mediante **Shaders de Computación**. Esto permite resolver el sistema de la red neuronal en paralelo:
$$\vec{y} = \sigma(\mathbf{W}\vec{x} + \vec{b})$$
Donde $\mathbf{W}$ es la matriz de pesos, $\vec{x}$ los datos de entrada, $\vec{b}$ el sesgo y $\sigma$ la función de activación (ej. ReLU). Millones de estas operaciones ocurren cada 16ms.

## 6. Integración Técnica: El Vínculo MediaPipe - Three.js

En tu implementación, el flujo de datos representa un ciclo completo de **Métodos Numéricos Aplicados**:

1. **Captura:** La cámara obtiene la matriz de píxeles $I$.
2. **Inferencia (MediaPipe):** Se resuelve la regresión para obtener los landmarks $\vec{P}_i$.
3. **Mapeo:** Se transforman las coordenadas normalizadas $[0,1]$ al espacio euclidiano $[-n, n]$.
4. **Transformación (Three.js):** Se actualizan las matrices de rotación y traslación de los objetos 3D.
5. **Render:** Se proyectan los objetos 3D de nuevo a 2D para el monitor.

### 6.1 Ejemplo de Lógica de Unión (Fragmento del código)
```javascript
// La regresión de MediaPipe entrega 'f' (landmarks)
const f = res.multiFaceLandmarks[0];
const le = f[33], re = f[263]; // Ojo Izquierdo y Derecho

// Cálculo de la distancia euclidiana (Norma L2) para la escala
const eyeDist = Math.abs(le.x - re.x) * 3;

// Aplicación de la transformación afín (Traslación)
glasses.position.set(centerX, centerY, -0.35);

### 6.2 Resumen Final de la Tecnología

| Proceso | Método Numérico / Matemático |
| :--- | :--- |
| **Detección de Objetos** | Convolución Discreta y Redes Neuronales Convolucionales (CNN). |
| **Landmarks (Puntos Clave)** | Regresión No-lineal de alta dimensionalidad sobre espacios latentes. |
| **Estabilidad de Señal** | Filtros de Paso Bajo adaptativos (Filtro One Euro). |
| **Alineación 3D** | Resolución del problema PnP (Perspective-n-Point) y Optimización de Reproyección. |
| **Rendimiento** | Cuantización de tensores y computación paralela en GPU. |

---

> **Conclusión de MediaPipe:** Esta tecnología representa la frontera de los métodos numéricos aplicados, donde la resolución de sistemas de ecuaciones no se hace de forma manual, sino que se delega a modelos de aprendizaje automático que optimizan billones de parámetros para transformar datos visuales brutos en información geométrica útil.

In [ ]:
from IPython.display import display, HTML

# Guardamos todo el código HTML/JS en una variable de texto de Python
codigo_html = """
<div id="main-container">
    <div class="panel">
      <button id="btn-glasses">👓 Lentes</button>
      <button id="btn-mustache">🥸 Bigote</button>
      <button id="btn-hat">🎩 Sombrero</button>
      <button id="btn-mole">🟤 Lunar</button>
      <button id="btn-points">🟢 FaceMesh</button>
    </div>

    <video id="videoElement" autoplay muted playsinline></video>
    <canvas id="canvasElement"></canvas>
    <div id="status">Cargando sistema...</div>
</div>

<script src="https://cdn.jsdelivr.net/npm/@mediapipe/face_mesh/face_mesh.js"></script>
<script src="https://cdn.jsdelivr.net/npm/@mediapipe/camera_utils/camera_utils.js"></script>

<style>
    #main-container {
        position: relative;
        width: 640px;
        height: 480px;
        background: black;
        margin: 0 auto;
        border: 2px solid #444;
        font-family: sans-serif;
        overflow: hidden;
    }
    video, canvas {
        position: absolute;
        top: 0; left: 0;
        width: 100%; height: 100%;
        object-fit: cover;
    }
    .panel {
        position: absolute;
        top: 10px; left: 10px;
        z-index: 50;
        display: flex;
        flex-direction: column;
        gap: 5px;
    }
    button {
        width: 140px;
        padding: 8px;
        cursor: pointer;
        font-weight: bold;
        border: 1px solid #999;
        border-radius: 4px;
        background: white;
        transition: background 0.2s;
    }
    /* Clase activa (Verde) */
    button.active {
        background-color: #76ff03 !important;
        border-color: #4caf50;
    }
    #status {
        position: absolute;
        bottom: 10px; left: 10px;
        color: white;
        background: rgba(0,0,0,0.5);
        padding: 5px;
        z-index: 40;
        pointer-events: none;
    }
</style>

<script>
(function() {
    // 1. Referencias
    const container = document.getElementById('main-container');
    if (!container) return; // Prevenir errores si se re-ejecuta
    
    const video = container.querySelector("#videoElement");
    const canvas = container.querySelector("#canvasElement");
    const ctx = canvas.getContext("2d");
    const statusDiv = container.querySelector("#status");

    // 2. Estado de los filtros
    const filters = {
        glasses: false,
        mustache: false,
        hat: false,
        mole: false,
        points: false
    };

    // 3. CONEXIÓN DE BOTONES
    const buttons = {
        'btn-glasses': 'glasses',
        'btn-mustache': 'mustache',
        'btn-hat': 'hat',
        'btn-mole': 'mole',
        'btn-points': 'points'
    };

    // Asignar eventos clic
    for (const [btnId, filterName] of Object.entries(buttons)) {
        const btn = container.querySelector("#"+btnId);
        if (btn) {
            btn.onclick = function() {
                filters[filterName] = !filters[filterName];
                // Feedback Visual
                if (filters[filterName]) {
                    btn.classList.add('active');
                } else {
                    btn.classList.remove('active');
                }
            };
        }
    }

    // 4. Ajustar Canvas
    function resize() {
        canvas.width = 640;
        canvas.height = 480;
    }
    resize();

    // 5. Configurar FaceMesh
    const faceMesh = new FaceMesh({
        locateFile: (file) => `https://cdn.jsdelivr.net/npm/@mediapipe/face_mesh/${file}`
    });

    faceMesh.setOptions({
        maxNumFaces: 1,
        refineLandmarks: true,
        minDetectionConfidence: 0.5,
        minTrackingConfidence: 0.5
    });

    faceMesh.onResults(onResults);

    // 6. Dibujar Resultados
    function onResults(res) {
        statusDiv.innerText = "Rostro detectado ✅";
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        
        // Dibujar imagen de video
        ctx.drawImage(res.image, 0, 0, canvas.width, canvas.height);

        if (res.multiFaceLandmarks && res.multiFaceLandmarks.length > 0) {
            const landmarks = res.multiFaceLandmarks[0];
            drawFilters(landmarks, canvas.width, canvas.height);
        } else {
            statusDiv.innerText = "Buscando rostro...";
        }
    }

    function drawFilters(f, w, h) {
        // Puntos clave
        const le = f[33];   // Ojo izq
        const re = f[263];  // Ojo der
        const lip = f[13];  // Labio
        const cheek = f[50];// Mejilla

        // Geometría
        const cx = (le.x + re.x) / 2 * w;
        const cy = (le.y + re.y) / 2 * h;
        const dx = (re.x - le.x) * w;
        const dy = (re.y - le.y) * h;
        const dist = Math.sqrt(dx*dx + dy*dy);

        if (filters.points) {
            ctx.fillStyle = "#00ff00";
            for (let p of f) {
                ctx.beginPath();
                ctx.arc(p.x * w, p.y * h, 1, 0, 2 * Math.PI);
                ctx.fill();
            }
        }

        if (filters.glasses) {
            ctx.strokeStyle = "black";
            ctx.lineWidth = 5;
            ctx.strokeRect(cx - dist*0.8, cy - dist*0.3, dist*0.7, dist*0.4);
            ctx.strokeRect(cx + dist*0.1, cy - dist*0.3, dist*0.7, dist*0.4);
            ctx.beginPath();
            ctx.moveTo(cx - dist*0.1, cy - dist*0.1);
            ctx.lineTo(cx + dist*0.1, cy - dist*0.1);
            ctx.stroke();
        }

        if (filters.mustache) {
            const mx = lip.x * w;
            const my = lip.y * h - dist * 0.15;
            ctx.strokeStyle = "black"; 
            ctx.lineWidth = dist * 0.2;
            ctx.lineCap = "round";
            ctx.beginPath();
            ctx.moveTo(mx - dist * 0.5, my + dist * 0.1);
            ctx.quadraticCurveTo(mx, my - dist * 0.2, mx + dist * 0.5, my + dist * 0.1);
            ctx.stroke();
        }

        if (filters.hat) {
            ctx.fillStyle = "#5d4037";
            ctx.fillRect(cx - dist, cy - dist * 1.8, dist * 2, dist * 0.8);
            ctx.fillRect(cx - dist * 1.4, cy - dist, dist * 2.8, dist * 0.2);
        }
        
        if (filters.mole) {
             ctx.fillStyle = "#3e2723";
             ctx.beginPath();
             ctx.arc(cheek.x*w, cheek.y*h, dist*0.1, 0, Math.PI*2);
             ctx.fill();
        }
    }

    // 7. Iniciar Cámara
    const camera = new Camera(video, {
        onFrame: async () => {
            await faceMesh.send({image: video});
        },
        width: 640,
        height: 480
    });

    statusDiv.innerText = "Solicitando cámara...";
    camera.start()
        .then(() => statusDiv.innerText = "Cámara lista.")
        .catch(err => statusDiv.innerText = "Error: " + err);

})();
</script>
"""

# Esta línea final es la que hace la magia de mostrar el HTML
display(HTML(codigo_html))